### Do some basic data sanitization using pandas, save data into CSV for sql lite database

1. Given `codefoobackend_cfgames.csv` was inspected for any irregularities. There did not seem to be any clear things (e.g. empty cells that should not be empty or if they were empty, I am not sure how I would have filled in data). If I knew more context (e.g. franchise fields must always be filled) about the data, I would try to fill them in (e.g. insert creators and publishers for each media item).
3. `STORING`: I chose SQLlite because it is small, light-weight and easy to use. Plus the given dataset was small. If it was bigger, I may have used MySQL or something else more robust.
3. `SANITIZING`: I sanitized using pandas library from Python since it was easier to manipulate. Might investigate the data more closely given more time using some helpful hints here: https://support.microsoft.com/en-us/office/top-ten-ways-to-clean-your-data-2844b620-677c-47a7-ac3e-c2e157d1db19. But from my glance initially, there did not seem anything glaringly wrong with the dataset.
4. `NORMALIZATION`: I think the right theoretical way to build the data base is to split an entity or object into its own table (e.g. 1 table for each "thing"). I started normalizing the data to make it do that. But for practicality, I created some tables just for easy usage (e.g. movies table). This would avoid costly joins (if the dataset was much bigger).
5. `INDEXING`: Once the data was in SQLlite database, I tried to index on what I think the most common queries will be for API which are: `short_name` and `name`. Maybe apply some considerations from links like this in the future: https://dataschool.com/sql-optimization/how-indexing-works/ or https://medium.com/the-software-firehose/how-to-choose-a-table-index-for-your-sql-database-d47715a35f34 


In [2]:
import pandas as pd
data = pd.read_csv('codefoobackend_cfgames.csv')

In [40]:
reviews = data[['id', 'name', 'short_name', 'created_at', 'updated_at', 'review_url','review_score']]
media = data[['id', 'name', 'short_description']]
media_metadata = data[['id', 'media_type', 'short_name', 'long_description', 'slug', 'genres', 'created_by', 'published_by', 'franchises', 'regions']]

# Views to make for easy access:
view_columns_to_keep = ['id', 'name', 'media_type', 'genres', 'created_by', 'published_by', 'short_description', 'franchises', 'regions']
preview = media.merge(media_metadata, left_on ='id', right_on='id')[view_columns_to_keep]
# Specific Views
movies = preview[preview['media_type'] == 'Movie']
games = preview[preview['media_type'] == 'Game']
comics = preview[preview['media_type'] == 'Comic']
shows = preview[preview['media_type'] == 'Show']

# Views:
views = {"movies": movies, "game": games, "comic": comics, "shows": shows}
tables = {"reviews": reviews, "media": media, "media_meta_data": media_metadata}

for file_name in views:
    df = views[file_name]
    df.to_csv("raw_csv_files/" + file_name + ".csv", index=False)
    
for file_name in tables:
    df = tables[file_name]
    df.to_csv("raw_csv_files/" + file_name + ".csv", index=False)
    

### Load data into a SQLlite database using:

1. .import FILE TABLE (https://www.sqlitetutorial.net/sqlite-import-csv/)

    Example SQL QUERY 
    ```
        CREATE TABLE IF NOT EXISTS "comics"(
        "id" TEXT,
        "name" TEXT,
        "media_type" TEXT,
        "genres" TEXT,
        "created_by" TEXT,
        "published_by" TEXT,
        "short_description" TEXT,
        "franchises" TEXT,
        "regions" TEXT
        );
        CREATE UNIQUE INDEX id_comics_idx ON comics(id);
        CREATE INDEX comics_name_idx ON comics(name);
    ```

2. I would see a couple of tables in SQLlite as listed below: 
    ```
    comic            games            media_meta_data  reviews        
    comics           media            movies           shows 
    ```
3. Indexes Created:
    * Create UNIQUE INDEX on all id columns for all tables: `CREATE UNIQUE INDEX id_media_meta_data_idx ON "media_metadata"(id);`
    * Create Index on name or short name column for all relevant tables: `CREATE INDEX media_meta_data_short_name_idx ON "media_metadata"(short_name);`
    
4. Save the database 